In [1]:
import requests
import pandas as pd
import json
import psycopg2


In [7]:
url = "https://dictionary.yandex.net/api/v1/dicservice.json/lookup"
headers = {"key":"dict.1.1.20190828T215834Z.0d8a3da5e08df3a0.85290ff28457230b81cb3b73720d2a0312ca14dc",
          "lang":"ru-fr","text":"book"}

In [5]:
url = "https://dictionary.yandex.net/api/v1/dicservice.json/lookup"
headers = {"key":"dict.1.1.20190828T215834Z.0d8a3da5e08df3a0.85290ff28457230b81cb3b73720d2a0312ca14dc",
              "lang":"ru-fr","text":"книга"}
ddd = requests.get(url, headers).json()
ddd

{'head': {},
 'def': [{'text': 'книга',
   'pos': 'noun',
   'gen': 'ж',
   'anm': 'неодуш',
   'tr': [{'text': 'livre',
     'pos': 'noun',
     'gen': 'm',
     'syn': [{'text': 'carnet', 'pos': 'noun', 'gen': 'm'},
      {'text': 'livret', 'pos': 'noun', 'gen': 'm'},
      {'text': 'imprimé', 'pos': 'noun', 'gen': 'm'},
      {'text': 'bouquin', 'pos': 'noun', 'gen': 'm'},
      {'text': 'journal', 'pos': 'noun', 'gen': 'm'},
      {'text': 'registre', 'pos': 'noun', 'gen': 'm'}],
     'mean': [{'text': 'книжка'}, {'text': 'брошюра'}, {'text': 'журнал'}],
     'ex': [{'text': 'священные книги', 'tr': [{'text': 'livres sacrés'}]},
      {'text': 'адресная книга', 'tr': [{'text': 'adresse carnet'}]}]},
    {'text': 'book', 'pos': 'noun', 'gen': 'm'},
    {'text': 'ouvrage',
     'pos': 'noun',
     'gen': 'm',
     'mean': [{'text': 'учебник'}],
     'ex': [{'text': 'книга под названием',
       'tr': [{'text': 'ouvrage intitulé'}]}]},
    {'text': 'cahier',
     'pos': 'noun',
     '

In [32]:
conn = psycopg2.connect(dbname='pgstage', user='linguist', password='eDQGK0GCStlYlHNV', host='192.168.122.183')
cursor = conn.cursor()


In [40]:
def get_word_forms(word):
    req = """SELECT DISTINCT
    content_words.word_id,content_words.word_lemma, content_words.word_value
    FROM content_words
    WHERE content_words.word_value !~ ('\W') AND
    (array_length(regexp_split_to_array(content_words.word_value, '[ ''-]'), 1) = 1) 
    and word_lemma != 0 and word_hash = calc_hash(' """ + word + """')"""
    cursor.execute(req)
    req_res = cursor.fetchone()
    if not req_res:
        print(word, " is not in db")
        return []
    else:
        #print(a[0])
        word_id = req_res[0]
    word_forms_request = """SELECT DISTINCT 
    content_words.word_value
    FROM content_words
    where word_lemma = """ + str(word_id) + """  and word_lemma != word_id """
    cursor.execute(word_forms_request)
    word_forms = []
    for word in cursor:
        #print(word[0])
        word_forms.append(word[0])
    return word_forms
get_word_forms("book")


['booker', 'booked', 'booking', 'books']

In [3]:
ddd = requests.get(url, headers).json()

In [75]:
ddd['def'][1]['tr'][0]

{'text': 'rezervovat',
 'pos': 'verb',
 'syn': [{'text': 'objednat', 'pos': 'verb'}]}

In [13]:
words = pd.read_csv("test_words.csv", header = None)
words.head()

,0
0,word
1,the
2,and
3,that
4,for


In [24]:
with open("./y_lookup/en_ru/word.json", 'w') as f:
    json.dump(ddd, f, indent = 4, ensure_ascii=False)

In [3]:
words = pd.read_csv("test_words.csv", header = None)
words.head()
wds = []
for word in words[0].items():
    print(word)
    wds.append(word[1])

(0, 'word')
(1, 'the')
(2, 'and')
(3, 'that')
(4, 'for')
(5, 'with')
(6, 'but')
(7, 'you')
(8, 'this')
(9, 'not')
(10, 'have')
(11, 'letter')
(12, 'plate')
(13, 'treat')
(14, 'dinner')
(15, 'necessary')
(16, 'go')
(17, 'secret')
(18, 'table')
(19, 'create')
(20, 'warm')
(21, 'America')
(22, 'nobody')
(23, 'entire')
(24, 'thousand')
(25, 'Belgium')
(26, 'anything')
(27, 'background')
(28, 'child')
(29, 'childhood')
(30, 'dependable')
(31, 'everything')
(32, 'grandchild')
(33, 'ketchup')
(34, 'point')
(35, 'strength')
(36, 'star')
(37, 'thing')
(38, 'coincide')
(39, 'coincidental')
(40, 'dance')
(41, 'countryside')
(42, 'break')
(43, 'mother')
(44, 'start')
(45, 'tip')
(46, 'apologise')
(47, 'come')
(48, 'curriculum')
(49, 'expertise')
(50, 'freeze')
(51, 'issue')
(52, 'peremptory')
(53, 'parent')
(54, 'member')
(55, 'band')
(56, 'order')
(57, 'hot')
(58, 'fly')
(59, 'lie')
(60, 'select')
(61, 'school')
(62, 'cafe')
(63, 'buy')
(64, 'tell')
(65, 'international')
(66, 'stand')
(67, 'fish'

In [4]:
wds.index('exhaustively')

99

In [9]:
count = 0 
for word in words[0].items():
    print (word[1])
    headers = {"key":"dict.1.1.20190828T215834Z.0d8a3da5e08df3a0.85290ff28457230b81cb3b73720d2a0312ca14dc",
          "lang":"en-ru","text":word[1]}
    ddd = requests.get(url, headers).json()
    save_loc = "./y_lookup/yandex_lookup_big_en_ru/" + word[1] + ".json"
    with open(save_loc, 'w') as f:
        json.dump(ddd, f, indent = 4, ensure_ascii=False)
    count += 1
    #if count > 1:break

word
the
and
that
for
with
but
you
this
not
have
letter
plate
treat
dinner
necessary
go
secret
table
create
warm
America
nobody
entire
thousand
Belgium
anything
background
child
childhood
dependable
everything
grandchild
ketchup
point
strength
star
thing
coincide
coincidental
dance
countryside
break
mother
start
tip
apologise
come
curriculum
expertise
freeze
issue
peremptory
parent
member
band
order
hot
fly
lie
select
school
cafe
buy
tell
international
stand
fish
benefit
intelligent
follow
journey
news
write
approbation
read
hear
feel
be
right
guru
beginner
collide
genetically
precedent
refugee
secular
pasta
beaver
tiger
lion
pen
coconut
dynamite
spoon
ocean
mug
shampoo
buzzer
exhaustively
apoplexy
submissively
typhus
ravioli
meeting
actually
earring
add
adjective
drawing
dream
dressed
driving
drum
dry
duck
advanced
during
earn
singular
easily
east
eighth
adventure
eighty
adverb
electric
advertisement
century
size
electricity
elephant
else
empty
engine
engineer
enough
enter
entrance
ad

In [36]:
count = 0 
for word in words[0].items():
    print (word[1])
    wordforms = get_word_forms(word[1].lower())
    for wf in wordforms:
        print(">>>", wf)
        headers = {"key":"dict.1.1.20190828T215834Z.0d8a3da5e08df3a0.85290ff28457230b81cb3b73720d2a0312ca14dc",
              "lang":"en-ru","text":wf}
        ddd = requests.get(url, headers).json()
        save_loc = "./y_lookup/en_ru/" + wf + ".json"
        with open(save_loc, 'w') as f:
            json.dump(ddd, f, indent = 4, ensure_ascii=False)
    count += 1
    if count > 100:break

word
>>> wording
>>> worded
>>> words
the
>>> thebed
>>> thest
>>> the bed
>>> thebest
>>> thes
and
>>> anding
>>> ands
>>> ood
>>> andren
>>> anded
>>> ander
that
>>> that led
>>> thats
>>> those
for
>>> forring
>>> fored
>>> fors
with
>>> withim
>>> withing
>>> withs
but
>>> buts
you
>>> youed
>>> yous
>>> youta
>>> youx
>>> youing
>>> youger
this
>>> thises
>>> this led
>>> these
not
>>> not bed
>>> notest
>>> notting
>>> nots
have
>>> have led
>>> haveta
>>> haveing
>>> had
>>> haved
>>> haves
>>> having
>>> havest
letter
>>> lettered
>>> letterd
>>> lettering
>>> letters
>>> lettera
plate
>>> plating
>>> plated
>>> plates
treat
>>> treats
>>> treated
>>> treaten
>>> treating
>>> treatest
dinner
>>> dinners
necessary
>>> necessarie
go
>>> gota
>>> gos
>>> gone
>>> went
>>> goed
>>> goes
>>> goren
>>> going
secret
>>> secrets
>>> secret
>>> secreter
table
>>> tables
>>> tabled
>>> tabling
>>> tabler
create
>>> creating
>>> creates
>>> createing
>>> created
warm
>>> warmest
>>> warme

TypeError: 'int' object is not iterable

# EN FR

In [42]:
count = 0 
for word in words[0].items():
    print (word[1])
    wordforms = get_word_forms(word[1].lower())
    for wf in wordforms:
        print(">>>", wf)
        headers = {"key":"dict.1.1.20190828T215834Z.0d8a3da5e08df3a0.85290ff28457230b81cb3b73720d2a0312ca14dc",
              "lang":"en-fr","text":wf}
        ddd = requests.get(url, headers).json()
        save_loc = "./y_lookup/en_fr/" + wf + ".json"
        with open(save_loc, 'w') as f:
            json.dump(ddd, f, indent = 4, ensure_ascii=False)
    count += 1
    if count > 100:break

word
>>> wording
>>> worded
>>> words
the
>>> thebed
>>> thest
>>> the bed
>>> thebest
>>> thes
and
>>> anding
>>> ands
>>> ood
>>> andren
>>> anded
>>> ander
that
>>> that led
>>> thats
>>> those
for
>>> forring
>>> fored
>>> fors
with
>>> withim
>>> withing
>>> withs
but
>>> buts
you
>>> youed
>>> yous
>>> youta
>>> youx
>>> youing
>>> youger
this
>>> thises
>>> this led
>>> these
not
>>> not bed
>>> notest
>>> notting
>>> nots
have
>>> have led
>>> haveta
>>> haveing
>>> had
>>> haved
>>> haves
>>> having
>>> havest
letter
>>> lettered
>>> letterd
>>> lettering
>>> letters
>>> lettera
plate
>>> plating
>>> plated
>>> plates
treat
>>> treats
>>> treated
>>> treaten
>>> treating
>>> treatest
dinner
>>> dinners
necessary
>>> necessarie
go
>>> gota
>>> gos
>>> gone
>>> went
>>> goed
>>> goes
>>> goren
>>> going
secret
>>> secrets
>>> secret
>>> secreter
table
>>> tables
>>> tabled
>>> tabling
>>> tabler
create
>>> creating
>>> creates
>>> createing
>>> created
warm
>>> warmest
>>> warme

In [10]:
count = 0 
for word in words[0].items():
    print (word[1])
    headers = {"key":"dict.1.1.20190828T215834Z.0d8a3da5e08df3a0.85290ff28457230b81cb3b73720d2a0312ca14dc",
          "lang":"en-fr","text":word[1]}
    ddd = requests.get(url, headers).json()
    save_loc = "./y_lookup/en_fr_whole_list/" + word[1] + ".json"
    with open(save_loc, 'w') as f:
        json.dump(ddd, f, indent = 4, ensure_ascii=False)
    count += 1

word
the
and
that
for
with
but
you
this
not
have
letter
plate
treat
dinner
necessary
go
secret
table
create
warm
America
nobody
entire
thousand
Belgium
anything
background
child
childhood
dependable
everything
grandchild
ketchup
point
strength
star
thing
coincide
coincidental
dance
countryside
break
mother
start
tip
apologise
come
curriculum
expertise
freeze
issue
peremptory
parent
member
band
order
hot
fly
lie
select
school
cafe
buy
tell
international
stand
fish
benefit
intelligent
follow
journey
news
write
approbation
read
hear
feel
be
right
guru
beginner
collide
genetically
precedent
refugee
secular
pasta
beaver
tiger
lion
pen
coconut
dynamite
spoon
ocean
mug
shampoo
buzzer
exhaustively
apoplexy
submissively
typhus
ravioli
meeting
actually
earring
add
adjective
drawing
dream
dressed
driving
drum
dry
duck
advanced
during
earn
singular
easily
east
eighth
adventure
eighty
adverb
electric
advertisement
century
size
electricity
elephant
else
empty
engine
engineer
enough
enter
entrance
ad

In [4]:
ddd

{'head': {},
 'def': [{'text': 'book',
   'pos': 'noun',
   'ts': 'bʊk',
   'tr': [{'text': 'книга',
     'pos': 'noun',
     'gen': 'ж',
     'syn': [{'text': 'книжка', 'pos': 'noun', 'gen': 'ж'},
      {'text': 'книжечка', 'pos': 'noun', 'gen': 'ж'},
      {'text': 'книжица', 'pos': 'noun', 'gen': 'ж'}],
     'mean': [{'text': 'volume'}, {'text': 'booklet'}],
     'ex': [{'text': 'new book', 'tr': [{'text': 'новая книга'}]},
      {'text': 'red data book', 'tr': [{'text': 'красная книга'}]},
      {'text': "children's book", 'tr': [{'text': 'детская книга'}]},
      {'text': 'personal address book',
       'tr': [{'text': 'личная адресная книга'}]},
      {'text': 'very good book', 'tr': [{'text': 'очень хорошая книга'}]},
      {'text': 'recently published book',
       'tr': [{'text': 'недавно опубликованная книга'}]},
      {'text': 'local phone book', 'tr': [{'text': 'телефонная книга'}]},
      {'text': 'book of mormon', 'tr': [{'text': 'книга мормона'}]},
      {'text': 'cheque

In [22]:
url = "https://dictionary.yandex.net/api/v1/dicservice.json/getLangs"
h = {"key":"dict.1.1.20190828T215834Z.0d8a3da5e08df3a0.85290ff28457230b81cb3b73720d2a0312ca14dc"}
d = requests.get(url, h).json()

In [ ]:
['be-be',
 'be-ru',
 'bg-ru',
 'cs-cs',
 'cs-en',
 'cs-ru',
 'da-en',
 'da-ru',
 'de-de',
 'de-en',
 'de-ru',
 'de-tr',
 'el-en',
 'el-ru',
 'en-cs',
 'en-da',
 'en-de',
 'en-el',
 'en-en',
 'en-es',
 'en-et',
 'en-fi',
 'en-fr',
 'en-it',
 'en-lt',
 'en-lv',
 'en-nl',
 'en-no',
 'en-pt',
 'en-ru',
 'en-sk',
 'en-sv',
 'en-tr',
 'en-uk',
 'es-en',
 'es-es',
 'es-ru',
 'et-en',
 'et-ru',
 'fi-en',
 'fi-ru',
 'fi-fi',
 'fr-fr',
 'fr-en',
 'fr-ru',
 'hu-hu',
 'hu-ru',
 'it-en',
 'it-it',
 'it-ru',
 'lt-en',
 'lt-lt',
 'lt-ru',
 'lv-en',
 'lv-ru',
 'mhr-ru',
 'mrj-ru',
 'nl-en',
 'nl-ru',
 'no-en',
 'no-ru',
 'pl-ru',
 'pt-en',
 'pt-ru',
 'ru-be',
 'ru-bg',
 'ru-cs',
 'ru-da',
 'ru-de',
 'ru-el',
 'ru-en',
 'ru-es',
 'ru-et',
 'ru-fi',
 'ru-fr',
 'ru-hu',
 'ru-it',
 'ru-lt',
 'ru-lv',
 'ru-mhr',
 'ru-mrj',
 'ru-nl',
 'ru-no',
 'ru-pl',
 'ru-pt',
 'ru-ru',
 'ru-sk',
 'ru-sv',
 'ru-tr',
 'ru-tt',
 'ru-uk',
 'sk-en',
 'sk-ru',
 'sv-en',
 'sv-ru',
 'tr-de',
 'tr-en',
 'tr-ru',
 'tt-ru',
 'uk-en',
 'uk-ru',
 'uk-uk',
 'zh-ru']